<a href="https://colab.research.google.com/github/vaishjah3/Assignments/blob/main/MNIST_handwritten_digits_feedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
input_size=784
hidden_size=400
output_size=10
epochs=10
batch_size=100
learning_rate=0.001

In [ ]:
train_dataset=datasets.MNIST(root='./data',
                             train=True,
                             transform=transforms.ToTensor(),
                             download=True
                        )
test_dataset=datasets.MNIST(root='./data',
                            transform=transforms.ToTensor(),
                            train=False
                            )

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 83428739.62it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 16382971.85it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 88165668.86it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2702202.66it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)


In [ ]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.fc1=nn.Linear(input_size, hidden_size)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_size, hidden_size)
    self.fc3=nn.Linear(hidden_size, output_size)
  def forward(self, x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.relu(out)
    out=self.fc3(out)
    return out

In [ ]:
net=Net(input_size, hidden_size, output_size)
CUDA=torch.cuda.is_available()
if CUDA:
  net=net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
for x,y in train_loader:
  print(x.size())
  break

torch.Size([100, 1, 28, 28])


In [ ]:
import cv2
import time


In [ ]:
#Train the network
for epoch in range(epochs):
  correct_train=0
  running_loss=0
  for i, (images, labels) in enumerate(train_loader):
    #image size from (100,1,28,28) to (100,768)
    images=images.view(-1, 784)
    if CUDA:
      images.cuda()
      labels.cuda()
    outputs=net(images)
    #print(outputs.size())
    #print(outputs)
    _, predicted=torch.max(outputs.data, 1) #first_ndex (max_value), 2nd index(index of max_value)
    #print(predicted)
    correct_train += (predicted==labels).sum()
    loss=criterion(outputs,labels)
    #print(type(loss))
    running_loss+=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%'.format
          (epoch+1, epochs, running_loss/len(train_loader), (100*correct_train.double()/len(train_dataset))))

print("DONE TRAINING!")



Streaming output truncated to the last 5000 lines.
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 61.195%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 61.360%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 61.525%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 61.692%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 61.858%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.025%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.190%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.355%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.522%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.688%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 62.853%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 63.020%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 63.185%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 63.348%
Epoch [2/10], Training Loss: 0.015, Training Accuracy: 63.515%
Epoc

In [ ]:
with torch.no_grad():
  correct=0
  for images, labels in test_loader:
    if CUDA:
      images.cuda()
      labels.cuda()
    images=images.view(-1, 28*28)
    outputs=net(images)
    _, predicted=torch.max(outputs.data, 1)
    correct+=(predicted==labels).sum().item()
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / len(test_dataset)))

Accuracy of the network on the 10000 test images: 98.15 %
